In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization, Dropout
import matplotlib.pyplot as plt

In [3]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

In [4]:
train_ds=tf.data.Dataset.from_tensor_slices((train_images,train_labels))
test_ds=tf.data.Dataset.from_tensor_slices((test_images,test_labels))
CLASS_NAMES= ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

FailedPreconditionError: Failed to allocate scratch buffer for device 0

In [ ]:
train_ds_size=tf.data.experimental.cardinality(train_ds).numpy()
test_ds_size=tf.data.experimental.cardinality(test_ds).numpy()
print('Train size:',train_ds_size)
print('Test size:',test_ds_size)

In [ ]:
def process_image(image, label):
  image = tf.image.per_image_standardization(image)
  return image, label

train_ds=(train_ds
          .map(process_image)
          .shuffle(buffer_size=train_ds_size)
          .batch(batch_size=32,drop_remainder=True)
         )
test_ds=(test_ds
          .map(process_image)
          .shuffle(buffer_size=test_ds_size)
          .batch(batch_size=32,drop_remainder=True)
          )

plt.figure(figsize=(30,30))
for i,(image,label) in enumerate(train_ds.take(20)):
    print(label)
    ax=plt.subplot(5,5,i+1)
    plt.imshow(image)
    plt.title(CLASS_NAMES[label.numpy()[0]])
    plt.axis('off')

In [ ]:
def VGG_block(num_convs, num_channels):
    block = Sequential()
    for _ in range(num_convs):
        block.add(Conv2D(filters=num_channels, kernel_size=3, padding='same', activation='relu'))
    block.add(MaxPooling2D(pool_size=2, strides=2))
    return block

def VGG_net(arch, lr=0.1, num_classes=10):
    model = Sequential()
    for (num_convs, num_channels) in arch:
        model.add(VGG_block(num_convs, num_channels))
    model.add(
        Sequential([
            Flatten(),
            Dense(4096,activation='relu'),
            Dropout(0.5),
            Dense(4096,activation='relu'),
            Dropout(0.5),
            Dense(num_classes, activation='softmax')
        ])
    )
    return model

In [ ]:
model = VGG_net([(1, 64), (1, 128), (2, 256), (2, 512), (2, 512)])
model.summary()